# 08. MatterGen - AI 기반 저탄소 시멘트 대체재 발견

## 연구 목적
MatterGen을 활용하여 기존 Top 5 산업폐기물 기반 후보를 보완하고,  
AI가 발견한 새로운 저탄소 시멘트 대체재를 제시합니다.

---

### 실행 계획

| Phase | 내용 | 모델 |
|:-----:|------|------|
| 1 | Top 5 기반 구조 다양화 | chemical_system_energy_above_hull |
| 2 | C-S-H 핵심 조성 탐색 | chemical_system_energy_above_hull |
| 3 | 특성 기반 역설계 | ml_bulk_modulus |

### 대상 화학 시스템

| 시스템 | 기반 | 목적 |
|--------|------|------|
| Ca-Si-Al-O | FlyAshC | Top 1 조성 다양화 |
| Ca-Si-Al-Fe-O | EAFSlag | Top 2 조성 다양화 |
| Ca-Si-O | C3S | 핵심 시멘트 조성 |
| Ca-Si-Mg-O | Slag | 대체 조성 탐색 |

## 1. 환경 설정

In [ ]:
import sys
import os
from pathlib import Path
import json
import time

# tqdm 텍스트 모드 강제 (위젯 오류 방지)
os.environ['TQDM_NOTEBOOK'] = 'false'
from tqdm import tqdm

# 프로젝트 경로 설정
PROJECT_ROOT = Path.cwd().parent.parent
MATTERGEN_REPO = PROJECT_ROOT / 'mattergen'
OUTPUT_DIR = PROJECT_ROOT / 'data' / 'mattergen'
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

# MatterGen 경로 추가
sys.path.insert(0, str(MATTERGEN_REPO))
os.chdir(MATTERGEN_REPO)

print(f"Project Root: {PROJECT_ROOT}")
print(f"MatterGen Repo: {MATTERGEN_REPO}")
print(f"Output Directory: {OUTPUT_DIR}")

In [ ]:
# 사용 가능한 체크포인트 확인
checkpoints_dir = MATTERGEN_REPO / 'checkpoints'

print("Available MatterGen Checkpoints:")
print("=" * 50)
for ckpt in sorted(checkpoints_dir.iterdir()):
    if ckpt.is_dir() and (ckpt / 'config.yaml').exists():
        ckpt_file = ckpt / 'checkpoints' / 'last.ckpt'
        size = ckpt_file.stat().st_size / 1e6 if ckpt_file.exists() else 0
        print(f"  - {ckpt.name} ({size:.0f} MB)")

## 2. MatterGen 실행 함수 정의

In [ ]:
from mattergen.generator import CrystalGenerator
from mattergen.common.utils.data_classes import MatterGenCheckpointInfo

def run_mattergen_generation(
    model_name: str,
    output_name: str,
    properties: dict,
    batch_size: int = 4,
    num_batches: int = 2,
    guidance_factor: float = 2.0
):
    """
    MatterGen 구조 생성 (Python API - 내부 tqdm 자동 표시)
    
    Parameters:
    -----------
    model_name : str
        체크포인트 이름 (예: 'chemical_system_energy_above_hull')
    output_name : str
        출력 폴더 이름
    properties : dict
        조건 속성 (예: {'chemical_system': 'Ca-Si-Al-O', 'energy_above_hull': 0.05})
    batch_size : int
        배치 크기 (기본값: 4)
    num_batches : int
        배치 수 (총 생성 수 = batch_size * num_batches)
    guidance_factor : float
        조건부 가이던스 강도 (기본값: 2.0)
    
    Returns:
    --------
    tuple: (output_path, structures)
    """
    output_path = OUTPUT_DIR / output_name
    output_path.mkdir(exist_ok=True)
    
    checkpoint_path = MATTERGEN_REPO / 'checkpoints' / model_name
    
    print(f"{'=' * 60}")
    print(f"MatterGen Generation: {output_name}")
    print(f"   Model: {model_name}")
    print(f"   Properties: {properties}")
    print(f"   Structures: {batch_size} x {num_batches} = {batch_size * num_batches}")
    print(f"{'=' * 60}\n")
    
    start_time = time.time()
    
    # CheckpointInfo 생성
    print("Loading model...")
    checkpoint_info = MatterGenCheckpointInfo(
        model_path=checkpoint_path,
        load_epoch="last"
    )
    
    # Generator 생성
    generator = CrystalGenerator(
        checkpoint_info=checkpoint_info,
        properties_to_condition_on=properties,
        batch_size=batch_size,
        num_batches=num_batches,
        diffusion_guidance_factor=guidance_factor
    )
    
    # 구조 생성 (내부 tqdm 진행 바 표시)
    print("Generating structures...")
    structures = generator.generate()
    
    # 결과 저장
    print("\nSaving results...")
    from ase.io import write
    from ase import Atoms
    
    saved_files = []
    for i, struct in enumerate(structures):
        cif_path = output_path / f"structure_{i:03d}.cif"
        # pymatgen Structure -> ASE Atoms 변환
        atoms = Atoms(
            symbols=[str(s) for s in struct.species],
            positions=struct.cart_coords,
            cell=struct.lattice.matrix,
            pbc=True
        )
        write(str(cif_path), atoms)
        saved_files.append(cif_path)
    
    elapsed = time.time() - start_time
    
    print(f"\n[SUCCESS] Time: {elapsed:.1f}s")
    print(f"Output: {output_path}")
    print(f"Generated {len(structures)} structures:")
    
    for f in saved_files:
        size = f.stat().st_size / 1024
        print(f"   - {f.name} ({size:.1f} KB)")
    
    return output_path, structures

print("Generation function defined.")

---

## Phase 1: Top 5 기반 구조 다양화

기존 스크리닝에서 선정된 Top 5 산업폐기물의 화학 조성을 기반으로  
안정적인 새로운 결정 구조를 생성합니다.

### Phase 1.1: Ca-Si-Al-O (FlyAshC 기반)

In [ ]:
# Phase 1.1: Ca-Si-Al-O 시스템 (FlyAshC 기반)
result_flyash = run_mattergen_generation(
    model_name='chemical_system_energy_above_hull',
    output_name='phase1_Ca_Si_Al_O',
    properties={
        'chemical_system': 'Ca-Si-Al-O',
        'energy_above_hull': 0.05  # 50 meV/atom 이하
    },
    batch_size=4,
    num_batches=2,
    guidance_factor=2.0
)

### Phase 1.2: Ca-Si-Al-Fe-O (EAFSlag 기반)

In [ ]:
# Phase 1.2: Ca-Si-Al-Fe-O 시스템 (EAFSlag 기반)
result_eafslag = run_mattergen_generation(
    model_name='chemical_system_energy_above_hull',
    output_name='phase1_Ca_Si_Al_Fe_O',
    properties={
        'chemical_system': 'Ca-Si-Al-Fe-O',
        'energy_above_hull': 0.05
    },
    batch_size=4,
    num_batches=2,
    guidance_factor=2.0
)

---

## Phase 2: C-S-H 핵심 조성 탐색

시멘트 수화 반응의 핵심 생성물인 C-S-H (Calcium Silicate Hydrate) 관련 조성을 탐색합니다.

### Phase 2.1: Ca-Si-O (핵심 시멘트 조성)

In [ ]:
# Phase 2.1: Ca-Si-O 시스템 (핵심 C-S-H 조성)
result_csh = run_mattergen_generation(
    model_name='chemical_system_energy_above_hull',
    output_name='phase2_Ca_Si_O',
    properties={
        'chemical_system': 'Ca-Si-O',
        'energy_above_hull': 0.0  # 열역학적으로 안정한 구조만
    },
    batch_size=4,
    num_batches=2,
    guidance_factor=2.0
)

### Phase 2.2: Ca-Si-Mg-O (Slag 기반 대체 조성)

In [ ]:
# Phase 2.2: Ca-Si-Mg-O 시스템 (Slag 기반)
result_slag = run_mattergen_generation(
    model_name='chemical_system_energy_above_hull',
    output_name='phase2_Ca_Si_Mg_O',
    properties={
        'chemical_system': 'Ca-Si-Mg-O',
        'energy_above_hull': 0.05
    },
    batch_size=4,
    num_batches=2,
    guidance_factor=2.0
)

---

## Phase 3: 특성 기반 역설계 (Inverse Design)

원하는 기계적 특성(Bulk Modulus)을 지정하여 해당 특성을 갖는 구조를 생성합니다.  
이것이 MatterGen의 핵심 기능인 **역설계(Inverse Design)** 입니다.

In [ ]:
# Phase 3: Bulk Modulus 기반 역설계
# 목표: 100 GPa 이상의 높은 강성을 가진 구조
result_inverse = run_mattergen_generation(
    model_name='ml_bulk_modulus',
    output_name='phase3_bulk_modulus_100',
    properties={
        'ml_bulk_modulus': 100  # 100 GPa
    },
    batch_size=4,
    num_batches=2,
    guidance_factor=2.0
)

---

## 4. 결과 요약

In [ ]:
# 생성된 모든 구조 요약
print("=" * 60)
print("MatterGen Generation Summary")
print("=" * 60)

for phase_dir in sorted(OUTPUT_DIR.iterdir()):
    if phase_dir.is_dir():
        cif_files = list(phase_dir.glob('*.cif'))
        print(f"\n{phase_dir.name}:")
        print(f"   - Generated: {len(cif_files)} structures")
        if cif_files:
            total_size = sum(f.stat().st_size for f in cif_files) / 1024
            print(f"   - Total size: {total_size:.1f} KB")

---

## 다음 단계

생성된 구조들은 **09_MatterGen_Validation.ipynb**에서 검증됩니다:

1. **CHGNet 에너지 계산** - 구조 안정성 평가
2. **구조 최적화** - 생성된 구조 relaxation
3. **Top 5 비교** - 기존 후보들과 성능 비교
4. **수화 시뮬레이션** - 선별된 구조의 수화 반응 테스트